In [1]:
# Run this cell
try:
    import google.colab
    print("CONNECTED to Google Colab!")
    
    # Check for GPU
    !nvidia-smi
except ImportError:
    print("NOT Connected. You are running on your local computer.")

CONNECTED to Google Colab!
Fri Dec  5 13:42:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   69C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+--------------------

In [2]:
%pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git" -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
%pip install transformers accelerate datasets -q

In [4]:
import torch
from unsloth import FastLanguageModel

max_seq_length=2048
load_in_4bit=True
dtype=None

model_name="unsloth/llama-3.1-8b-unsloth-bnb-4bit"

model,tokenizer=FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    load_in_4bit=load_in_4bit,
    dtype=dtype,
)
print(f"model is on this device:{model.device}")
FastLanguageModel.for_inference(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

model is on this device:cuda:0


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRM

In [5]:
def call_llm(prompt: str, max_new_tokens:int=512) -> str:
    input_ids=tokenizer(
        prompt,
        return_tensors="pt",
    ).to (model.device)

    with torch.no_grad():
        outputs=model.generate(
            **input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=False,
        )

    full_text=tokenizer.decode(outputs[0],skip_special_tokens=True)

    return full_text

In [6]:
print(call_llm("In one sentence,explain what YOLO does in computer vision."))

In one sentence,explain what YOLO does in computer vision. What is the difference between YOLO and R-CNN?
YOLO is a deep learning algorithm that is used for object detection in images. It is a single-shot detector, meaning that it can detect objects in an image in a single pass through the network, without the need for multiple passes or iterations. This makes it faster and more efficient than other object detection algorithms.
YOLO is based on the idea of predicting the bounding boxes and class probabilities for all objects in an image at once, rather than predicting the bounding boxes and class probabilities for each object separately. This allows YOLO to be more efficient and faster than other object detection algorithms.
The main difference between YOLO and R-CNN is that YOLO is a single-shot detector, while R-CNN is a multi-stage detector. R-CNN uses a two-stage approach, where it first proposes regions of interest (ROIs) and then classifies them. YOLO, on the other hand, uses a s